In [38]:
import requests
from bs4 import BeautifulSoup
import time    # 랜덤 딜레이시
import random  # 랜덤 딜레이시
import re # 정규 표현식
import pandas as pd # 데이터프레임 사용

# ----------------------
# 1. 상수 정의 (PC 버전)
# ----------------------
BASE_URL = "https://gall.dcinside.com/mgallery/board/lists"
ARTICLE_BASE_URL = "https://gall.dcinside.com"

# User-Agent 목록 정의(랜덤선택)
USER_AGENT_LIST = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/140.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/139.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 14_2_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
]


def get_regular_post_data(gallery_id: str, search_keyword: str = "", start_page: int = 1, end_page: int = 3) -> pd.DataFrame:
    """
    PC 갤러리 페이지에서 게시물의 제목과 내용을 추출하여 DataFrame으로 반환합니다.
    """
    
    df = pd.DataFrame(columns=['Title', 'Content', 'GalleryID', 'URL'])
    
    for i in range(start_page, end_page + 1):
        
        # ----------------------
        # 1단계: 목록 페이지 요청 및 파싱
        # ----------------------
        
        params = {'id': gallery_id, 'page': i}

        # 검색 주소 조립 시 필요한 파라미터 정의
        # ex) https://gall.dcinside.com/mgallery/board/lists/?id=warship&s_type=search_subject_memo&s_keyword=알래스카
        if search_keyword:
            # PC 검색 파라미터 사용
            params['search_pos'] = ''
            params['s_type'] = 'search_subject_memo'
            params['s_keyword'] = search_keyword

        # User-Agent 설정
        user_agent = random.choice(USER_AGENT_LIST)
        headers = {'User-Agent': user_agent}

        # try-except
        try:
            print(f"--- 갤러리 목록 페이지 {i} 요청 중 ---")
            response = requests.get(BASE_URL, params=params, headers=headers, timeout=10)
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"목록 페이지 {i} 요청 실패: {e}. 다음 페이지로 이동합니다.")
            time.sleep(random.uniform(2, 4))
            continue

        # lxml 파서 사용(HTML 대신)
        soup = BeautifulSoup(response.content, 'lxml')
        
        # 글 목록 구조: <tbody> 내의 <tr>
        article_list = soup.find('tbody').find_all('tr', {'data-type': ['icon_pic', 'icon_txt']})
        
        # 기본 공지, 광고글 필터링
        filtered_articles = []
        for tr_item in article_list:
            writer_tag = tr_item.find('td', class_='gall_writer')
            is_operator_post = writer_tag and writer_tag.get('user_name') == '운영자'
            is_notice = tr_item.get('data-type') == 'icon_notice'
            
            if not is_operator_post and not is_notice:
                filtered_articles.append(tr_item)
                
        if not filtered_articles:
             print(f"페이지 {i}에서 유효한 일반 게시물이 없습니다. 크롤링 종료.")
             break 


        # ----------------------
        # 2단계: 개별 게시물 접근 및 내용 추출 
        # ----------------------
        for tr_item in filtered_articles:
            
            title_tag = tr_item.find('a', href=True)
            if not title_tag: continue

            title_raw = title_tag.text.strip()
            relative_url = title_tag['href']
            
            # href 절대 경로/상대 경로 모두 대응 (없어도 솔직히 문제 없을듯?)
            if relative_url.startswith('http'):
                full_url = relative_url
            else:
                full_url = ARTICLE_BASE_URL + relative_url

            # 랜덤 딜레이
            time.sleep(random.uniform(3, 5))
            
            # 게시물 본문 요청
            try:
                print(f"   -> 게시물 요청: {title_raw[:20]}...")
                article_user_agent = random.choice(USER_AGENT_LIST)
                article_headers = {'User-Agent': article_user_agent}
                article_response = requests.get(full_url, headers=article_headers, timeout=10)
                article_response.raise_for_status()
            except requests.exceptions.RequestException as e:
                print(f"   -> 게시물 요청 실패 ({full_url}): {e}")
                continue
            
            article_soup = BeautifulSoup(article_response.content, 'lxml') # lxml 사용

            # 본문 추출 클래스: 'write_div'
            article_contents_tag = article_soup.find('div', class_='write_div')
            article_contents = ""
            if article_contents_tag:
                # 텍스트만 추출하고 불필요한 공백 제거
                article_contents = BeautifulSoup(str(article_contents_tag), "lxml").text.strip()
            
            # ----------------------
            # 3단계: 데이터 클리닝 및 저장
            # ----------------------
            
            # 제목과 게시글에서 url 제거
            pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$\-@\.&+:/?=]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
            repl = ''
            title_clean = re.sub(pattern=pattern, repl=repl, string=title_raw).strip()
            article_contents_clean = re.sub(pattern=pattern, repl=repl, string=article_contents).strip()
            
            # '- dc official App' 제거
            article_contents_clean = article_contents_clean.replace('- dc official App', '').strip()
            
            
            if article_contents_clean:
                
                new_row = pd.DataFrame([{
                    'Title': title_clean,
                    'Content': article_contents_clean,
                    'GalleryID': gallery_id,
                    'URL': full_url
                }])
                
                df = pd.concat([df, new_row], ignore_index=True)
                
    return df

In [39]:
# Cell 2: 함수 실행 및 결과 확인

# 1. 갤러리 ID와 검색어 설정
gallery_id_target = 'warship'
search_keyword_target = '알래스카'
start_page_num = 1
end_page_num = 2 # 1페이지부터 3페이지까지 수집

# 2. 함수 호출
results_df = get_regular_post_data(
    gallery_id=gallery_id_target, 
    search_keyword=search_keyword_target, 
    start_page=start_page_num, 
    end_page=end_page_num
)

# 3. 결과 출력 및 확인
print(f"최종 수집된 게시물 수: {len(results_df)}개")

# 주피터 노트북은 head()를 호출하면 DataFrame을 테이블 형태로 예쁘게 출력합니다.
results_df.head()

# CSV 저장 (선택 사항)
if not results_df.empty:
    results_df.to_csv("dcinside_data.csv", index=False, encoding="utf-8-sig")
    print("\n데이터가 dcinside_data.csv 파일로 저장되었습니다.")

--- 갤러리 목록 페이지 1 요청 중 ---
   -> 게시물 요청: 알래스카 피해 복구반 어떻게 써야함?...
   -> 게시물 요청: 리코 잘타는 법 좀 ㅠㅠㅠ...
   -> 게시물 요청: 블프상자 25개로 알래스카 사이판먹었...
   -> 게시물 요청: 알래스카 메추리 사는이유...
   -> 게시물 요청: 결국 알래스카 삿다...
   -> 게시물 요청: 알래스카 지르기전 마지막 질문, 진짜...
   -> 게시물 요청: PTS 작전 밸런스 조정 하는듯?...
   -> 게시물 요청: 25% 골드쿠폰 소진용 골쉽 추천점...
   -> 게시물 요청: 블프 아타고, 마인츠는 관짝 들어갔다...
   -> 게시물 요청: 결국 키어사지도 샀다...
   -> 게시물 요청: 원래 알래스카 살랬는데...
   -> 게시물 요청: 근데 알래스카 있으면 좋음...
   -> 게시물 요청: 소신발언)알래스카 이제 필구급은 아님...
   -> 게시물 요청: 뉴비가 대순타면 불타뒤진다고 알래스카...
   -> 게시물 요청: 중고 뉴비인데  알래스카 진짜 좆냐?...
   -> 게시물 요청: 알래스카가 진짜 명품이네...
   -> 게시물 요청: 쉽붕이 순양 입문으로 알래스카 샀는데...
   -> 게시물 요청: 조지아집탄으로도 포격연습이 잘 안되더...
   -> 게시물 요청: 뉴비 9티어 배 고민이요ㅜ...
   -> 게시물 요청: 근데 알래스카 현금가격 올라도...
--- 갤러리 목록 페이지 2 요청 중 ---
   -> 게시물 요청: 만토이펠이 알래스카 상위호환 같은데...
   -> 게시물 요청: 알래스카에 환상 가진 뉴비들이 많네...
   -> 게시물 요청: 아니씹 알래스카 가격 올렸네...
   -> 게시물 요청: 알래스카 가격 또 올랐네 아 ㅋㅋㅋㅋ...
   -> 게시물 요청: 뉴비 알래스카 철갑탄 질문좀...
   -> 게시물 요청: 혹시 1주전 워쉽 블프 가격목록 아시...
   -> 게시물 요청: 클전 밴픽을 게임사가 자체적으로 박는...
